In [12]:
import requests
import aiohttp
import asyncio
import nest_asyncio


In [13]:
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Define the base URL and headers
URL = "https://chess-api.com/v1"
headers = {"content-type": "application/json"}

In [24]:
def get_centi_pawn_evaluation(fen: str) -> float:
    """
    Get the centi pawn evaluation of a given FEN string.

    Args:
        fen (str): FEN string of the chess position.

    Returns:
        float: The centi pawn evaluation of the chess position.
    """
    payload = {"fen": fen, "depth": 18}
    response = requests.post(URL, headers=headers, json=payload)

    return float(response.json()["centipawns"])

In [25]:
print(get_centi_pawn_evaluation("rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 1 2"))
print(get_centi_pawn_evaluation("rn1qkb1r/ppp1pppp/3p4/3nP3/3P2b1/5N2/PPP2PPP/RNBQKB1R w KQkq - 2 5"))

71.0
85.0


In [32]:
async def get_centi_pawn_evaluation(session, fen: str) -> float:
    """
    Asynchronously get the centi pawn evaluation of a given FEN string.

    Args:
        session (aiohttp.ClientSession): The session object to reuse connections.
        fen (str): FEN string of the chess position.

    Returns:
        float: The centi pawn evaluation of the chess position.
    """
    payload = {"fen": fen, "depth": 18, "maxThinkingTime": 100,}
    async with session.post(URL, headers=headers, json=payload) as response:
        data = await response.json()
        return float(data["centipawns"])

In [33]:
async def fetch_all_evaluations(fens: list) -> list:
    """
    Fetch centi pawn evaluations for a list of FEN strings concurrently.

    Args:
        fens (list): List of FEN strings to evaluate.

    Returns:
        list: List of centi pawn evaluations.
    """
    async with aiohttp.ClientSession() as session:
        tasks = [get_centi_pawn_evaluation(session, fen) for fen in fens]
        return await asyncio.gather(*tasks)

In [34]:
fens = [
        "rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 1 2",
        "rn1qkb1r/ppp1pppp/3p4/3nP3/3P2b1/5N2/PPP2PPP/RNBQKB1R w KQkq - 2 5"
    ]
evaluations = asyncio.run(fetch_all_evaluations(fens))  # Or call asyncio.run() directly in a script
print(evaluations)

{'text': 'Move e4 → e5 (e5): [0.79]. White is winning. Depth 18.', 'captured': False, 'promotion': False, 'isCapture': False, 'isPromotion': False, 'isCastling': False, 'fen': 'rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 1 2', 'type': 'bestmove', 'depth': 18, 'move': 'e4e5', 'eval': 0.79, 'centipawns': '79', 'mate': None, 'continuationArr': ['f6d5', 'd2d4', 'd7d6', 'c2c4', 'd5b6', 'f2f4', 'd6e5', 'f4e5', 'b8c6', 'c1e3', 'c8f5', 'b1c3', 'e7e6', 'g1f3', 'f8e7', 'f1e2', 'e8g8', 'e1g1', 'f7f6', 'e5f6', 'e7f6', 'd1d2', 'd8e7', 'a1d1', 'a8d8', 'e3f2', 'f5g4'], 'debug': 'info depth 18 seldepth 34 multipv 1 score cp 79 nodes 1276803 nps 35466750 hashfull 29 tbhits 0 time 36 pv e4e5 f6d5 d2d4 d7d6 c2c4 d5b6 f2f4 d6e5 f4e5 b8c6 c1e3 c8f5 b1c3 e7e6 g1f3 f8e7 f1e2 e8g8 e1g1 f7f6 e5f6 e7f6 d1d2 d8e7 a1d1 a8d8 e3f2 f5g4', 'winChance': 57.22126153120401, 'taskId': 'nz00cl7dp', 'turn': 'w', 'color': 'w', 'piece': 'p', 'from': 'e4', 'to': 'e5', 'san': 'e5', 'flags': 'n', 'lan': 'e4e5', 'fr